In [309]:
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing.imputation import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score
# from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import os
import random
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import roc_auc_score

In [310]:
def makeUserTarget(df, user):
    aux = []
    for _, row in df.iterrows():
        if row.our_data == 0:
            if(np.floor(row.SessionID/100).astype(int) == user):
                aux.append(1)
            else:
                aux.append(0)
        else:
            if 'Diego' in row.SessionID and user == 'Diego':
                aux.append(1)
            elif 'Toto' in row.SessionID and user == 'Toto':
                aux.append(1)
            elif (('Jo' in row.SessionID) or ('jo' in row.SessionID)) and user == 'Jo':
                aux.append(1)
            else:
                aux.append(0)
    df['target'] = aux

In [311]:
def separateIntoTrainAndTest(df, test_sessions = [],randomn = 10):
    targetSessionIds = list(df[df.target == 1].SessionID.unique())
    targetChoosenSession = random.choice(targetSessionIds)
    notTargetSessionIds = list(df[df.target == 0].SessionID.unique())
    notTargetChoosenSessions = test_sessions
    for i in range(randomn):
        notTargetChoosenSession = random.choice(notTargetSessionIds) 
        notTargetChoosenSessions.append(notTargetChoosenSession)
        notTargetSessionIds.remove(notTargetChoosenSession)
    df_train = df[(df.SessionID != targetChoosenSession) & (~df.SessionID.isin(notTargetChoosenSessions))]
    df_test = df[(df.SessionID == targetChoosenSession) | (df.SessionID.isin(notTargetChoosenSessions))]
    
    return df_train, df_test

In [312]:
def getOurDataUsers(row):
    if 'Diego' in row.SessionID :
        return('Diego')
    elif 'Toto' in row.SessionID:
        return('Toto')
    elif ('Jo' in row.SessionID) or ('jo' in row.SessionID):
        return('Jo')

In [313]:
#file_name = 'aggregated_features'
file_name = '10secs_aggregated_features'
our_data_file_name = 'our_data_aggregated_features'
to_drop = []
df = pd.read_csv('./' + file_name + '.csv')
our_data_df = pd.read_csv('./' + our_data_file_name + '.csv')
df['our_data'] = 0
our_data_df['our_data'] = 1

In [314]:
users = (np.floor(df.SessionID/100).astype(int)).unique()
our_data_users = our_data_df.apply(getOurDataUsers,axis=1)

In [315]:
df = df.append(our_data_df,ignore_index=True)

In [316]:
df = df.dropna()

In [317]:
sessionsToDrop = ['Diego01']

In [318]:
df = df[~df.SessionID.isin(sessionsToDrop)]

In [319]:
#user = random.choice(users)
user = 'Jo'
makeUserTarget(df, user)

In [320]:
#to_drop =  ['our_data','Contact_size_mean','Contact_size_std'] 
#to_drop =  ['our_data','Mag_Z_mean','Mag_X_mean','Mag_Y_mean','Mag_Y_std','Mag_Z_std','Mag_X_std']
to_drop =  ['our_data','Mag_Z_mean','Mag_X_mean','Mag_Y_mean','Mag_Y_std','Mag_Z_std','Mag_X_std','Contact_size_mean','Pressure_mean','Pressure_std','Contact_size_std']
#to_drop =  ['our_data','Mag_Z_mean','Mag_X_mean','Mag_Y_mean','Mag_Y_std','Mag_Z_std','Mag_X_std','Pressure_mean','Pressure_std']

In [321]:
# clf = xgb.XGBClassifier(n_estimators=90, max_depth=9, random_state=31, colsample_bytree=0.6, colsample_bylevel=0.5, learning_rate=0.11, subsample=0.9)
# clf = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=31)
def create_network():
    network = Sequential()
    network.add(Dense(int(len(x_train.columns)), input_dim=len(x_train.columns) , activation = 'relu'))
    network.add(Dense(14, activation = 'relu'))
    network.add(Dense(12, activation = 'relu'))
    network.add(Dense(9, activation = 'relu'))
    network.add(Dense(6, activation = 'relu'))
    network.add(Dense(3, activation = 'relu'))
    network.add(Dense(1, activation = 'sigmoid'))
    network.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics=['accuracy'])
    return network
clf = create_network()


In [322]:
df_train, df_test = separateIntoTrainAndTest(df,test_sessions=['oscar1','Pedro1','Carlos1','Andres1'],randomn=1)

In [323]:
df_train = df_train.set_index(["SessionID", 'WindowNumber']).drop(to_drop, axis=1)
df_test = df_test.set_index(["SessionID", 'WindowNumber']).drop(to_drop, axis=1)

In [324]:
x_train, y_train = df_train.drop("target", axis=1), df_train["target"]

In [325]:
clf.fit(x = x_train, y = y_train, epochs = 5, batch_size = 32, validation_split = 0.1)

Train on 44736 samples, validate on 4971 samples
Epoch 1/5
44736/44736 [==============================] - 5s 116us/step - loss: 1.1535e-05 - acc: 1.0000 - val_loss: 0.6341 - val_acc: 0.9238
Epoch 2/5
44736/44736 [==============================] - 4s 91us/step - loss: 2.3928e-06 - acc: 1.0000 - val_loss: 0.6445 - val_acc: 0.9238
Epoch 3/5
44736/44736 [==============================] - 4s 89us/step - loss: 1.5847e-06 - acc: 1.0000 - val_loss: 0.6517 - val_acc: 0.9238
Epoch 4/5
44736/44736 [==============================] - 4s 90us/step - loss: 1.2074e-06 - acc: 1.0000 - val_loss: 0.6575 - val_acc: 0.9238
Epoch 5/5
44736/44736 [==============================] - 4s 93us/step - loss: 9.8384e-07 - acc: 1.0000 - val_loss: 0.6623 - val_acc: 0.9238


In [326]:
x_test, y_test = df_test.drop("target", axis=1), df_test["target"]

In [327]:
predict = clf.predict_proba(x_test)
wut = []
for row in predict:
    a = [(1-row[0]).tolist()]
    a.append(row[0].tolist())
    wut.append(a)
predict = np.array(wut)
predict

array([[9.99977839e-01, 2.21608498e-05],
       [9.99999797e-01, 2.03208558e-07],
       [9.99991157e-01, 8.84311612e-06],
       [9.99999992e-01, 7.87872612e-09],
       [9.99998049e-01, 1.95081998e-06],
       [1.00000000e+00, 3.56838309e-10],
       [1.00000000e+00, 6.00031750e-31],
       [9.99994623e-01, 5.37734559e-06],
       [9.99998845e-01, 1.15512603e-06],
       [9.99998985e-01, 1.01459341e-06],
       [9.99738315e-01, 2.61685112e-04],
       [9.99999859e-01, 1.40741420e-07],
       [9.99981806e-01, 1.81939904e-05],
       [9.99987165e-01, 1.28354905e-05],
       [1.00000000e+00, 1.88599460e-25],
       [9.99998175e-01, 1.82539088e-06],
       [9.99996898e-01, 3.10172823e-06],
       [9.99989841e-01, 1.01587048e-05],
       [1.00000000e+00, 2.01345559e-14],
       [1.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 9.88096557e-17],
       [1.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 2.72422869e-16],
       [1.00000000e+00, 2.81517453e-12],
       [1.000000

In [328]:
clf.predict_proba(x_test)[:]

array([[2.21608498e-05],
       [2.03208558e-07],
       [8.84311612e-06],
       [7.87872612e-09],
       [1.95081998e-06],
       [3.56838309e-10],
       [6.00031750e-31],
       [5.37734559e-06],
       [1.15512603e-06],
       [1.01459341e-06],
       [2.61685112e-04],
       [1.40741420e-07],
       [1.81939904e-05],
       [1.28354905e-05],
       [1.88599460e-25],
       [1.82539088e-06],
       [3.10172823e-06],
       [1.01587048e-05],
       [2.01345559e-14],
       [0.00000000e+00],
       [9.88096557e-17],
       [0.00000000e+00],
       [2.72422869e-16],
       [2.81517453e-12],
       [1.80680512e-23],
       [3.09215761e-13],
       [9.81123209e-14],
       [0.00000000e+00],
       [1.26758460e-24],
       [1.59367436e-10],
       [4.87166265e-22],
       [2.38248853e-14],
       [0.00000000e+00],
       [6.03859251e-17],
       [3.03410621e-22],
       [0.00000000e+00],
       [4.65919986e-25],
       [0.00000000e+00],
       [2.13515790e-16],
       [8.96494318e-14],


In [329]:
np.array(map(lambda x: x[0], clf.predict_proba(x_test)))

array([2.21608498e-05, 2.03208558e-07, 8.84311612e-06, 7.87872612e-09,
       1.95081998e-06, 3.56838309e-10, 6.00031750e-31, 5.37734559e-06,
       1.15512603e-06, 1.01459341e-06, 2.61685112e-04, 1.40741420e-07,
       1.81939904e-05, 1.28354905e-05, 1.88599460e-25, 1.82539088e-06,
       3.10172823e-06, 1.01587048e-05, 2.01345559e-14, 0.00000000e+00,
       9.88096557e-17, 0.00000000e+00, 2.72422869e-16, 2.81517453e-12,
       1.80680512e-23, 3.09215761e-13, 9.81123209e-14, 0.00000000e+00,
       1.26758460e-24, 1.59367436e-10, 4.87166265e-22, 2.38248853e-14,
       0.00000000e+00, 6.03859251e-17, 3.03410621e-22, 0.00000000e+00,
       4.65919986e-25, 0.00000000e+00, 2.13515790e-16, 8.96494318e-14,
       0.00000000e+00, 1.13854701e-23, 0.00000000e+00, 1.37703890e-10,
       3.94206783e-20, 3.33829707e-20, 4.93690137e-28, 0.00000000e+00,
       1.71487146e-13, 1.18650127e-38, 4.67227056e-11, 8.12284584e-23,
       1.74269386e-21, 1.24862356e-23, 1.01120273e-12, 2.58622784e-20,
      

In [330]:
df_predict = pd.DataFrame(np.array(map(lambda x: x[0], clf.predict_proba(x_test))), columns=["prob"], index=x_test.index)

In [331]:
df_socorro = df_predict.merge(df_test[['target']], right_index=True, left_index=True)

In [332]:
df_socorro[df_socorro.target == 0].reset_index().groupby('SessionID').mean()

,WindowNumber,prob,target
SessionID,,,
53950212,8.500000,1.936969e-05,0
Andres1,28.466667,1.104270e-11,0
Carlos1,38.510638,1.058684e-04,0
Pedro1,36.274194,2.029329e-03,0
oscar1,6.928571,2.225274e-04,0


In [333]:
df_socorro[df_socorro.target == 1].reset_index().groupby('SessionID').mean()

,WindowNumber,prob,target
SessionID,,,
jo9,24.5,0.008979,1


In [334]:
print(clf.feature_importances_)
# plt.figure()

AttributeError: 'Sequential' object has no attribute 'feature_importances_'

In [ ]:
i_pred_proba = clf.predict_proba(x_test)
auc = roc_auc_score(y_test, i_pred_proba[:, 1])